In [79]:
import sys
import math
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
%config IPcompletor.greedy = True
from sklearn.svm import LinearSVC, SVC
print("Python: {}".format(sys.version))
from imblearn.over_sampling import SMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, mean_squared_error,recall_score

Python: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [85]:
train_set = pd.read_csv("tcdml1920-rec-click-pred--training.csv", index_col = 0,error_bad_lines=True)#385688
test_set = pd.read_csv("tcdml1920-rec-click-pred--test.csv", index_col = 0,error_bad_lines=False)#9146
test_set.dropna(how="all", inplace=True)

### NULL values

In [86]:
train_set.replace(["\\N", "nA", "Not provided", "*unknown*"], np.nan, inplace=True)
test_set.replace(["\\N", "nA", "Not provided", "*unknown*"], np.nan, inplace=True)

In [87]:
temp = train_set[(train_set["set_clicked"] == 1) | (train_set.time_recs_viewed.notnull())]

In [88]:
temp = temp.drop(columns = ["time_recs_viewed"])
test_set = test_set.drop(columns = ["time_recs_viewed"])

In [89]:
for i in temp.select_dtypes(include=['object']):
    target = temp.groupby(i)["set_clicked"].agg(pd.Series.mean)
    temp[i] = temp[i].map(target)
    test_set[i] = test_set[i].map(target)

In [90]:
temp[temp.columns]=temp[temp.columns].fillna(temp.mean().iloc[0])
target = temp.pop("set_clicked")
test_set[test_set.columns]=test_set[test_set.columns].fillna(test_set.mean().iloc[0])
pp= test_set.pop("set_clicked")

In [92]:
temp.isnull().sum()

user_id                             0
session_id                          0
query_identifier                    0
query_word_count                    0
query_char_count                    0
query_detected_language             0
query_document_id                   0
document_language_provided          0
year_published                      0
number_of_authors                   0
abstract_word_count                 0
abstract_char_count                 0
abstract_detected_language          0
first_author_id                     0
num_pubs_by_first_author            0
organization_id                     0
application_type                    0
item_type                           0
request_received                    0
hour_request_received               0
response_delivered                  0
rec_processing_time                 0
app_version                         0
app_lang                            0
user_os                             0
user_os_version                     0
user_java_ve

In [93]:
model1 = RandomForestClassifier()
model2 = LogisticRegression()
model3 = SVC()
model4 = DecisionTreeClassifier()

model1.fit(temp, target)
model2.fit(temp, target)
model3.fit(temp, target)
model4.fit(temp, target)

y_pred1 = model1.predict(test_set)
y_pred2 = model2.predict(test_set)
y_pred3 = model3.predict(test_set)
y_pred4 = model4.predict(test_set)

print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred2, return_counts=True)).T)
print(np.array(np.unique(y_pred3, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)

C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[   0 9065]
 [   1   80]]
[[   0 9145]]
[[   0 9131]
 [   1   14]]
[[   1 9145]]


In [96]:
submit["set_clicked"] = y_pred1

In [ ]:
train_1cbf = train_set[(train_set.organization_id == 1) & (train_set.algorithm_class == 'content_based_filtering')]
train_1 = train_set[(train_set.organization_id == 1) & (train_set.algorithm_class != 'content_based_filtering')]
train_2cbf = train_set[(train_set.organization_id != 1) & (train_set.algorithm_class == 'content_based_filtering')]
train_2 = train_set[(train_set.organization_id != 1) & (train_set.algorithm_class != 'content_based_filtering')]
train_4 = train_set[train_set.organization_id == 4]
train_8 = train_set[train_set.organization_id == 8]

In [ ]:
test_1cbf = test_set[(test_set.organization_id == 1) & (test_set.algorithm_class == 'content_based_filtering')]
test_1 = test_set[(test_set.organization_id == 1) & (test_set.algorithm_class != 'content_based_filtering')]
test_2cbf = test_set[(test_set.organization_id != 1) & (test_set.algorithm_class == 'content_based_filtering')]
test_2 = test_set[(test_set.organization_id != 1) & (test_set.algorithm_class != 'content_based_filtering')]
test_4 = test_set[test_set.organization_id == 4]
test_8 = test_set[test_set.organization_id == 8]

In [ ]:
train_1 = train_1.drop(columns =['algorithm_class','app_lang','app_version','application_type','cbf_parser','hour_request_received','item_type', 'local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])
train_1cbf = train_1cbf.drop(columns =['algorithm_class','app_lang','app_version','application_type','hour_request_received','item_type', 'local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])
test_1 = test_1.drop(columns =['algorithm_class','app_lang','app_version','application_type','cbf_parser','hour_request_received','item_type', 'local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])
test_1cbf = test_1cbf.drop(columns =['algorithm_class','app_lang','app_version','application_type','hour_request_received','item_type', 'local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])

In [ ]:
train_2 = train_2.drop(columns =['abstract_char_count', 'algorithm_class','app_lang','app_version','application_type','hour_request_received','local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])
train_2cbf = train_2cbf.drop(columns =['abstract_word_count', 'algorithm_class','app_lang','app_version','application_type','hour_request_received','local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])
test_2 = test_2.drop(columns =['abstract_char_count', 'algorithm_class','app_lang','app_version','application_type','hour_request_received','local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])
test_2cbf = test_2cbf.drop(columns =['abstract_word_count', 'algorithm_class','app_lang','app_version','application_type','hour_request_received','local_time_of_request','number_of_recs_in_set','organization_id','query_char_count','query_detected_language','query_identifier','request_received','response_delivered','search_abstract','search_keywords','search_title','timezone_by_ip','user_id', 'session_id','abstract_detected_language','rec_processing_time', 'time_recs_recieved', 'time_recs_displayed'])

In [ ]:
train_1.dropna(thresh= train_1[train_1["set_clicked"] == 0].shape[0]*0.4, axis=1, inplace = True)
train_2.dropna(thresh= train_2.shape[0]*0.4, axis=1, inplace = True)
train_1cbf.dropna(thresh= train_1cbf.shape[0]*0.4, axis=1, inplace = True)
train_2cbf.dropna(thresh= train_2cbf.shape[0]*0.4, axis=1, inplace = True)
train_4.dropna(thresh= train_4.shape[0]*0.4, axis=1, inplace = True)
train_8.dropna(thresh= train_8.shape[0]*0.4, axis=1, inplace = True)

In [ ]:
print(train_1.shape)
print(train_2.shape)
print(train_1cbf.shape)
print(train_2cbf.shape)
print(train_4.shape)
print(train_8.shape)

In [ ]:
test_1 = test_1[list(train_1.columns)]
test_2 = test_2[list(train_2.columns)]
test_1cbf = test_1cbf[list(train_1cbf.columns)]
test_2cbf = test_2cbf[list(train_2cbf.columns)]
test_4 = test_4[list(train_4.columns)]
test_8 = test_8[list(train_8.columns)]

In [ ]:
train_1[train_1.columns]=train_1[train_1.columns].fillna(train_1.mode().iloc[0])
train_2[train_2.columns]=train_2[train_2.columns].fillna(train_2.mode().iloc[0])
train_1cbf[train_1cbf.columns]=train_1cbf[train_1cbf.columns].fillna(train_1cbf.mode().iloc[0])
train_2cbf[train_2cbf.columns]=train_2cbf[train_2cbf.columns].fillna(train_2cbf.mode().iloc[0])
train_4[train_4.columns]=train_4[train_4.columns].fillna(train_4.mode().iloc[0])
train_8[train_8.columns]=train_8[train_8.columns].fillna(train_8.mode().iloc[0])
test_1[test_1.columns]=test_1[test_1.columns].fillna(train_1.mode().iloc[0])
test_1cbf[test_1cbf.columns]=test_1cbf[test_1cbf.columns].fillna(train_1cbf.mode().iloc[0])
test_2[test_2.columns]=test_2[test_2.columns].fillna(train_2.mode().iloc[0])
test_2cbf[test_2cbf.columns]=test_2cbf[test_2cbf.columns].fillna(train_2cbf.mode().iloc[0])
test_4[test_4.columns]=test_4[test_4.columns].fillna(train_4.mode().iloc[0])
test_8[test_8.columns]=test_8[test_8.columns].fillna(train_8.mode().iloc[0])

### To Numeric

In [ ]:
numeric_cols_8 = ["abstract_word_count","query_word_count","query_char_count","query_document_id","abstract_char_count","hour_request_received","local_hour_of_request","recommendation_algorithm_id_used","clicks"]
numeric_cols_4 = ["query_word_count","query_char_count","hour_request_received","recommendation_algorithm_id_used","clicks"]
numeric_cols_1 = ["query_word_count","local_hour_of_request","recommendation_algorithm_id_used"]
numeric_cols_1cbf = ["query_word_count","local_hour_of_request","recommendation_algorithm_id_used"]
numeric_cols_2 = ["query_word_count", "query_document_id", "abstract_word_count", "local_hour_of_request","recommendation_algorithm_id_used","clicks"]
numeric_cols_2cbf = ["query_word_count", "query_document_id", "local_hour_of_request","recommendation_algorithm_id_used","clicks"]

In [ ]:
for i in numeric_cols_1:
    train_1[i] = pd.to_numeric(train_1[i], errors='coerce')
    test_1[i] = pd.to_numeric(test_1[i], errors='coerce')
for i in numeric_cols_4:
    train_4[i] = pd.to_numeric(train_4[i], errors='coerce')
    test_4[i] = pd.to_numeric(test_4[i], errors='coerce')
for i in numeric_cols_8:
    train_8[i] = pd.to_numeric(train_8[i], errors='coerce')
    test_8[i] = pd.to_numeric(test_8[i], errors='coerce')
for i in numeric_cols_1cbf:
    train_1cbf[i] = pd.to_numeric(train_1cbf[i], errors='coerce')
    test_1cbf[i] = pd.to_numeric(test_1cbf[i], errors='coerce')
for i in numeric_cols_2:
    train_2[i] = pd.to_numeric(train_2[i], errors='coerce')
    test_2[i] = pd.to_numeric(test_2[i], errors='coerce')
for i in numeric_cols_2cbf:
    train_2cbf[i] = pd.to_numeric(train_2cbf[i], errors='coerce')
    test_2cbf[i] = pd.to_numeric(test_2cbf[i], errors='coerce')

In [ ]:
test_1.index = test_1.index.astype("int")
test_1cbf.index = test_1cbf.index.astype("int")
test_2.index = test_2.index.astype("int")
test_2cbf.index = test_2cbf.index.astype("int")
test_4.index = test_4.index.astype("int")
test_8.index = test_8.index.astype("int")

### Encoding
- One hot
- Target
- Label

In [ ]:
test_1.dropna(how="all", inplace = True)
test_1cbf.dropna(how="all", inplace = True)
test_2.dropna(how="all", inplace = True)
test_2cbf.dropna(how="all", inplace = True)
test_4.dropna(how="all", inplace = True)
test_8.dropna(how="all", inplace = True)

In [ ]:
for i in train_1.select_dtypes(include=['object']):
    target = train_1.groupby(i)["set_clicked"].agg(pd.Series.mean)
    train_1[i] = train_1[i].map(target)
    test_1[i] = test_1[i].map(target)
for i in train_1cbf.select_dtypes(include=['object']):
    target = train_1cbf.groupby(i)["set_clicked"].agg(pd.Series.mean)
    train_1cbf[i] = train_1cbf[i].map(target)
    test_1cbf[i] = test_1cbf[i].map(target)
for i in train_2.select_dtypes(include=['object']):
    target = train_2.groupby(i)["set_clicked"].agg(pd.Series.mean)
    train_2[i] = train_2[i].map(target)
    test_2[i] = test_2[i].map(target)
for i in train_2cbf.select_dtypes(include=['object']):
    target = train_2cbf.groupby(i)["set_clicked"].agg(pd.Series.mean)
    train_2cbf[i] = train_2cbf[i].map(target)
    test_2cbf[i] = test_2cbf[i].map(target)

for i in train_4.select_dtypes(include=['object']):
    target = train_4.groupby(i)["set_clicked"].agg(pd.Series.mean)
    train_4[i] = train_4[i].map(target)
    test_4[i] = test_4[i].map(target)
for i in train_8.select_dtypes(include=['object']):
    target = train_8.groupby(i)["set_clicked"].agg(pd.Series.mean)
    train_8[i] = train_8[i].map(target)
    test_8[i] = test_8[i].map(target)

In [ ]:
test_1[test_1.columns]=test_1[test_1.columns].fillna(test_1.mean().iloc[0])
test_1cbf[test_1cbf.columns]=test_1cbf[test_1cbf.columns].fillna(test_1cbf.mean().iloc[0])
test_2[test_2.columns]=test_2[test_2.columns].fillna(test_2.mean().iloc[0])
test_2cbf[test_2cbf.columns]=test_2cbf[test_2cbf.columns].fillna(test_2cbf.mean().iloc[0])
test_4[test_4.columns]=test_4[test_4.columns].fillna(test_4.mean().iloc[0])
test_8[test_8.columns]=test_8[test_8.columns].fillna(test_8.mean().iloc[0])

In [ ]:
target_1 = train_1.pop("set_clicked")
test_1.drop(columns=["set_clicked"], inplace =True)
print("Target: {}, Training: {}".format(target_1.shape, train_1.shape))
target_1cbf = train_1cbf.pop("set_clicked")
test_1cbf.drop(columns=["set_clicked"], inplace =True)
print("Target: {}, Training: {}".format(target_1cbf.shape, train_1cbf.shape))
target_2 = train_2.pop("set_clicked")
test_2.drop(columns=["set_clicked"], inplace =True)
print("Target: {}, Training: {}".format(target_2.shape, train_2.shape))
target_2cbf = train_2cbf.pop("set_clicked")
test_2cbf.drop(columns=["set_clicked"], inplace =True)
print("Target: {}, Training: {}".format(target_2cbf.shape, train_2cbf.shape))
target_4 = train_4.pop("set_clicked")
test_4.drop(columns=["set_clicked"], inplace =True)
print("Target: {}, Training: {}".format(target_4.shape, train_4.shape))
target_8 = train_8.pop("set_clicked")
test_8.drop(columns=["set_clicked"], inplace =True)
print("Target: {}, Training: {}".format(target_8.shape, train_8.shape))

### SMOTE


In [18]:
smt = SVMSMOTE()
# train_1, target_1 = smt.fit_sample(train_1, target_1)
# train_4, target_4 = smt.fit_sample(train_4,target_4)
train_8, target_8 = smt.fit_sample(train_8,target_8)

smt = ADASYN()
train_1, target_1 = smt.fit_sample(train_1, target_1)
train_4, target_4 = smt.fit_sample(train_4,target_4)
# train_8, target_8 = smt.fit_sample(train_8,target_8)

### NearMiss

In [20]:
nr = NearMiss()
# train_1, target_1 = nr.fit_sample(train_1, target_1)
# train_4, target_4 = nr.fit_sample(train_4,target_4)
# train_8, target_8 = nr.fit_sample(train_8,target_8)

### Bagging

In [22]:
from sklearn.ensemble import BaggingClassifier
clf1 = BaggingClassifier()
clf4 = BaggingClassifier()
clf8 = BaggingClassifier()
clf1cbf = BaggingClassifier()
clf2 = BaggingClassifier()
clf2cbf = BaggingClassifier()
clf1.fit(train_1, target_1)
clf1cbf.fit(train_1cbf, target_1cbf)
clf2.fit(train_2, target_2)
clf2cbf.fit(train_2cbf, target_2cbf)
clf4.fit(train_4, target_4)
clf8.fit(train_8, target_8)
y_pred1 = clf1.predict(test_1)
y_pred1cbf = clf1cbf.predict(test_1cbf)
y_pred2 = clf2.predict(test_2)
y_pred2cbf = clf2cbf.predict(test_2cbf)
y_pred4 = clf4.predict(test_4)
y_pred8 = clf8.predict(test_8)
print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred1cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred2, return_counts=True)).T)
print(np.array(np.unique(y_pred2cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)
print(np.array(np.unique(y_pred8, return_counts=True)).T)

[[   0 1379]]
[[   0 3797]]
[[   0 1308]]
[[   0 2661]]
[[   0 3029]]
[[  0 940]]


### Logistic Regression

In [23]:
clf1 = LogisticRegression()
clf1cbf = LogisticRegression()
clf2 = LogisticRegression()
clf2cbf = LogisticRegression()
clf4 = LogisticRegression()
clf8 = LogisticRegression()
clf1.fit(train_1, target_1)
clf1cbf.fit(train_1cbf, target_1cbf)
clf2.fit(train_2, target_2)
clf2cbf.fit(train_2cbf, target_2cbf)
clf4.fit(train_4, target_4)
clf8.fit(train_8, target_8)
y_pred1 = clf1.predict(test_1)
y_pred1cbf = clf1cbf.predict(test_1cbf)
y_pred2 = clf2.predict(test_2)
y_pred2cbf = clf2cbf.predict(test_2cbf)
y_pred4 = clf4.predict(test_4)
y_pred8 = clf8.predict(test_8)
print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred1cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred2, return_counts=True)).T)
print(np.array(np.unique(y_pred2cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)
print(np.array(np.unique(y_pred8, return_counts=True)).T)

C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-package

[[   0 1379]]
[[   0 3797]]
[[   0 1308]]
[[   0 2661]]
[[   0 3029]]
[[  0 940]]


C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Decision Trees

In [24]:
dt_1 = DecisionTreeClassifier()
dt_4 = DecisionTreeClassifier()
dt_8 = DecisionTreeClassifier()
dt_1.fit(train_1, target_1)
dt_4.fit(train_4, target_4)
dt_8.fit(train_8, target_8)
y_pred1 = dt_1.predict(test_1)
y_pred4 = dt_4.predict(test_4)
y_pred8 = dt_8.predict(test_8)

print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)
print(np.array(np.unique(y_pred8, return_counts=True)).T)

[[   0 1379]]
[[   0 3029]]
[[  0 940]]


### Random Forest

In [25]:
clf1 = RandomForestClassifier()
clf4 = RandomForestClassifier()
clf8 = RandomForestClassifier()
clf1cbf = RandomForestClassifier()
clf2 = RandomForestClassifier()
clf2cbf = RandomForestClassifier()
clf1.fit(train_1, target_1)
clf1cbf.fit(train_1cbf, target_1cbf)
clf2.fit(train_2, target_2)
clf2cbf.fit(train_2cbf, target_2cbf)
clf4.fit(train_4, target_4)
clf8.fit(train_8, target_8)
y_pred1 = clf1.predict(test_1)
y_pred1cbf = clf1cbf.predict(test_1cbf)
y_pred2 = clf2.predict(test_2)
y_pred2cbf = clf2cbf.predict(test_2cbf)
y_pred4 = clf4.predict(test_4)
y_pred8 = clf8.predict(test_8)
print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred1cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred2, return_counts=True)).T)
print(np.array(np.unique(y_pred2cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)
print(np.array(np.unique(y_pred8, return_counts=True)).T)

C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 t

[[   0 1379]]
[[   0 3797]]
[[   0 1308]]
[[   0 2661]]
[[   0 3029]]
[[  0 940]]


C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### SVC

In [ ]:
clf1 = SVC(kernel="poly")
clf4 = SVC(kernel="linear")
clf8 = SVC(kernel="linear")
clf1cbf = SVC(kernel="poly")
clf2 = SVC(kernel="linear")
clf2cbf = SVC(kernel="linear")
clf1.fit(train_1, target_1)
clf1cbf.fit(train_1cbf, target_1cbf)
clf2.fit(train_2, target_2)
clf2cbf.fit(train_2cbf, target_2cbf)
clf4.fit(train_4, target_4)
clf8.fit(train_8, target_8)
y_pred1 = clf1.predict(test_1)
y_pred1cbf = clf1cbf.predict(test_1cbf)
y_pred2 = clf2.predict(test_2)
y_pred2cbf = clf2cbf.predict(test_2cbf)
y_pred4 = clf4.predict(test_4)
y_pred8 = clf8.predict(test_8)
print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred1cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred2, return_counts=True)).T)
print(np.array(np.unique(y_pred2cbf, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)
print(np.array(np.unique(y_pred8, return_counts=True)).T)

C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\agarwalv\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


### XGBoost

In [24]:
clf1 = xgb.XGBClassifier(objective="binary:logistic", learning_rate = 0.15)
clf4 = xgb.XGBClassifier(objective="binary:logistic", learning_rate = 0.08)
clf8 = xgb.XGBClassifier(objective="binary:logistic", learning_rate = 0.05)
test_1 = test_1.values
test_4 = test_4.values
test_8 = test_8.values
clf1.fit(train_1, target_1)
clf4.fit(train_4, target_4)
clf8.fit(train_8, target_8)
y_pred1 = clf1.predict(test_1)
y_pred4 = clf4.predict(test_4)
y_pred8 = clf8.predict(test_8)
print(np.array(np.unique(y_pred1, return_counts=True)).T)
print(np.array(np.unique(y_pred4, return_counts=True)).T)
print(np.array(np.unique(y_pred8, return_counts=True)).T)

[[   0 5176]]
[[   0 3029]]
[[  0 940]]


- abstract_char_count
- rec_processing_time
- query_char_count
- user_os
- clicks
- session_id

### Submission

In [28]:
pred_1 = pd.DataFrame(y_pred1, index=test_1.index)
pred_1cbf = pd.DataFrame(y_pred1cbf, index=test_1cbf.index)
pred_2 = pd.DataFrame(y_pred2, index=test_2.index)
pred_2cbf = pd.DataFrame(y_pred2cbf, index=test_2cbf.index)
pred_4 = pd.DataFrame(y_pred4, index=test_4.index)
pred_8 = pd.DataFrame(y_pred8, index=test_8.index)

In [30]:
pred = pred_1.append(pred_1cbf.append(pred_2.append(pred_2cbf))).rename(columns={0:"set_clicked"})

In [43]:
# pred = pred_1.append(pred_4.append(pred_8)).rename(columns={0:"set_clicked"})

In [95]:
submit = pd.read_csv("tcdml1920-rec-click-pred--submission file.csv", index_col= 0)

In [35]:
pred["set_clicked"].value_counts()

0    9071
1      74
Name: set_clicked, dtype: int64

In [97]:
submit.to_csv(r"prediction.csv", index = True)